In [ ]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

In [ ]:
## Load dataset
N = 20
data_path = data_dir + "/gmm_test/gmm_%d" % N
Data = []
data = torch.from_numpy(np.load(data_path + '/obs_%d.npy' % N)).float()
Data.append(data)
## Train Parameters
NUM_EPOCHS = 500
K = 3 
D = 2
SAMPLE_SIZE = 10
BATCH_SIZE = 40
CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:0')
PATH = 'vae-%dpts-%dsamples' % (N, SAMPLE_SIZE)
Train_Params = (NUM_EPOCHS, K, D, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH) 
## Model Parameters
NUM_HIDDEN_GLOBAL = 32
NUM_HIDDEN_LOCAL = 32
LEARNING_RATE =  1e-4

In [ ]:
## model initialization
from global_lstm_no_ss import *
from local_enc import *

vae_eta = LSTM_eta(K, D, BATCH_SIZE, SAMPLE_SIZE, NUM_HIDDEN_GLOBAL, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
vae_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
if CUDA:
    with torch.cuda.device(DEVICE):
        vae_eta.cuda()
        vae_z.cuda()
optimizer =  torch.optim.Adam(list(vae_z.parameters()) + list(vae_eta.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (vae_eta, vae_z)

In [ ]:
from vae import EUBO ## objective for baseline
from training_vae import train_vae ## training function for baseline
train_vae(models, EUBO, optimizer, Data, Train_Params)

In [ ]:
torch.save(os_eta.state_dict(), '../weights/vae-eta-%s' + PATH)
torch.save(os_z.state_dict(), '../weights/vae-z-%s' + PATH)